<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import psycopg2
import warnings

from projectutils.print_report import print_report

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
# Установление подключения
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [4]:
# Создаем пустой словарь для отчетов по результатам выполнения заданий
reports = {}

# Избавляемся от лишних предупреждений
warnings.simplefilter(action='ignore', category=UserWarning)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = '''
select
    count(v.id) vacancies_cnt
from vacancies v
'''
# результат запроса
vacancies_cnt = pd.read_sql_query(query_3_1, connection).iloc[0]['vacancies_cnt']
print(f'Количество вакансий в базе: {vacancies_cnt}')
reports['report_3_1'] = '''
В масштабах всего сервиса HeadHunter 49197 вакансий - это только частичная выборка.
Будем надеяться, что она репрезентативная, то есть в ней в равных пропорциях
присутствуют записи разных классов объектов 🙂. 
'''

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = '''
select
    count(e.id) employers_cnt
from employers e
'''
# результат запроса
employers_cnt = pd.read_sql_query(query_3_2, connection).iloc[0]['employers_cnt']
print(f'Количество работодателей в базе: {employers_cnt}')

reports['report_3_2'] = f'''
Учитывая, что общее количество юридических лиц и индивидуальных
предпринимателей в 2021 году достигло пордка 7 млн, и это только
в России, {employers_cnt} работодателей на этом фоне выглядят
более чем скромно. Это тоже частичная выборка. 🤔 Интересно будет
проверить её на пропуски и насколько она релевантна выборке
вакансий.
'''

Количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [7]:
# текст запроса
query_3_3 = '''
select
    count(a.id) areas_cnt
from areas a
'''
# результат запроса
areas_cnt = pd.read_sql_query(query_3_3, connection).iloc[0]['areas_cnt']
print(f'Количество регионов в базе: {areas_cnt}')

reports['report_3_3'] = f'''
В России насчитывается 630 городских округов в то время как количество
городов (регионов) в базе составляет {areas_cnt}, из чего можно сделать
предположение, что в базе присутствуют не только регионы России, но и 
других стран 🌏.
'''

Количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [8]:
# текст запроса
query_3_4 = '''
select
    count(i.id) industries_cnt
from industries i
'''
# результат запроса
industries_cnt = pd.read_sql_query(query_3_4, connection).iloc[0]['industries_cnt']
print(f'Количество сфер деятельности в базе: {industries_cnt}')

reports['report_3_4'] = '''
Относительно количества сфер деятельности трудно сказать что либо определенное.
В открытых источниках не удалось отыскать количество таких сфер в каталоге HH 😢.
'''

Количество сфер деятельности в базе: 294


***

In [9]:
# выводы по предварительному анализу данных
print_report(reports, 'Выводы по предварительному анализу данных.', 3)

<h1>Раздел проекта: 3. Выводы по предварительному анализу данных.</h1><br>
    <div class='report'>
    <br><h4>
В масштабах всего сервиса HeadHunter 49197 вакансий - это только частичная выборка.
Будем надеяться, что она репрезентативная, то есть в ней в равных пропорциях
присутствуют записи разных классов объектов 🙂. 
</h4><br><h4>
Учитывая, что общее количество юридических лиц и индивидуальных
предпринимателей в 2021 году достигло пордка 7 млн, и это только
в России, 23501 работодателей на этом фоне выглядят
более чем скромно. Это тоже частичная выборка. 🤔 Интересно будет
проверить её на пропуски и насколько она релевантна выборке
вакансий.
</h4><br><h4>
В России насчитывается 630 городских округов в то время как количество
городов (регионов) в базе составляет 1362, из чего можно сделать
предположение, что в базе присутствуют не только регионы России, но и 
других стран 🌏.
</h4><br><h4>
Относительно количества сфер деятельности трудно сказать что либо определенное.
В открытых источниках не удалось отыскать количество таких сфер в каталоге HH 😢.
</h4></div>

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [10]:
# текст запроса
query_4_1 = '''
select
	a.name area,
	count(v.id) vac_cnt
from vacancies v
	join areas a on v.area_id=a.id
group by a.id
order by 2 desc
'''

# результат запроса
vac_cnt_by_area = pd.read_sql_query(query_4_1, connection)
display(vac_cnt_by_area.head(5))

# визуализация ТОП-5 регионов по количеству вакансий
fig = px.bar(
    vac_cnt_by_area.head(5),
    x='area', y='vac_cnt', color='area',
    title='Топ 5 регионов по количеству вакансий',
    labels={
		'area': 'Регион',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.show()

# формируем отчет об инсайтах
msc_vac_cnt = vac_cnt_by_area.iloc[0]['vac_cnt']
spb_vac_cnt = vac_cnt_by_area.iloc[1]['vac_cnt']
ratio = round(msc_vac_cnt / spb_vac_cnt)
reports['report_4_1'] = f'''
В разрезе распределения ваксансий по регионам следующие инсайты:
<ol>
<li>Москва, ожидаемо, на первом месте 👍🏻</li>
<li>В ТОП-5 регионов кроме столиц (включая историческую) России,
Беларуси и Казахстана вошла научная столица России - Новосибирск.
Поздравим ребят 🥳;</li>
<li>Количество вакансий в Санкт-Петебурге в {ratio} раза меньше,
чем в Москве. Питер - расти быстрее 💪🏻!</li>
<ol>
'''

,area,vac_cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [11]:
# текст запроса
query_4_2 = '''
select count(v.id) vac_cnt
from vacancies v
where v.salary_from is not NULL or v.salary_to is not NULL
'''

# результат запроса
vac_cnt = pd.read_sql_query(query_4_2, connection).iloc[0]['vac_cnt']
print(f'Количество вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой: {vac_cnt}')

ratio = vac_cnt / (vacancies_cnt - vac_cnt)

reports['report_4_2'] = f'''
Соотношение количества вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой к
количству вакансий, в которых зарплата не указана вообще: {ratio:.2%}.
Получается что практически в половине случаев работодатели не указывают
в объявлении о вакансии ни нижнюю, ни верхнюю границы заработной платы 🙄.
Полагаю, что компании могут делать это с двумя целями:
<ol>
<li>Мотивировать соискателя на взаимодействие с HR-службой через вопрос: "А сколько буду получать 🤔?";</li>
<li>Не раскрывать своей политики в отношении формирования фонда оплаты труда и планов мотивации
конкурентам 🙊.</li>
<ol>
'''

Количество вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [12]:
# текст запроса
query_4_3 = '''
select 
	avg(v.salary_from) lower_salary_mean,
	avg(v.salary_to) upper_salary_mean
from vacancies v
'''

# результат запроса
salaries = pd.read_sql_query(query_4_3, connection)

lower_salary_mean = round(salaries.iloc[0]['lower_salary_mean'])
print(f'Среднее значение нижней границы зарплаты: {lower_salary_mean}')

upper_salary_mean = round(salaries.iloc[0]['upper_salary_mean'])
print(f'Среднее значение верхней границы зарплаты: {upper_salary_mean}')

ratio = upper_salary_mean / lower_salary_mean

reports['report_4_3'] = f'''
Верхняя граница заработной платы практически в {ratio:.2f} раз выше, чем нижняя 👈🏻.
Можно сделать вывод, что среднестатистически работодатели предполагают возможность
роста заработной платы сотрудника на {ratio-1:.0%} относительно начальной зарплаты
со временем. Но как известно, средние оценки - просто математический выкрутас,
который больше нацелен на сокращение количества информации, нежели на далеко идущие
выводы 🥸.
'''

Среднее значение нижней границы зарплаты: 71065
Среднее значение верхней границы зарплаты: 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [13]:
# текст запроса
query_4_4 = '''
select 
	v.schedule||'—'||v.employment sched_empl_comp,
	count(v.id) vac_cnt
from vacancies v
group by v.schedule, v.employment
order by 2 desc
'''

# результат запроса
sched_empl_cnt = pd.read_sql_query(query_4_4, connection)
frst_schd_empl_cmp = sched_empl_cnt.iloc[0]['sched_empl_comp']
scnd_schd_empl_cmp = sched_empl_cnt.iloc[1]['sched_empl_comp']
print(f'На втором месте по популярности сочетания типа рабочего графика и типа трудоустройства стоит: {scnd_schd_empl_cmp}')

# визуализация ТОП-5 по популярности сочетаний типа рабочего графика и типа трудоустройства
fig = px.bar(
    sched_empl_cnt.head(5),
    x='sched_empl_comp', y='vac_cnt', color='sched_empl_comp',
    title='ТОП-5 по популярности сочетаний типа рабочего графика и типа трудоустройства',
    labels={
		'sched_empl_comp': 'График-занятость',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.show()

# вывод отчета об интересном инсайте
frst_schd_empl_cnt = sched_empl_cnt.iloc[0]['vac_cnt']
scnd_schd_empl_cnt = sched_empl_cnt.iloc[1]['vac_cnt']
ratio = round(frst_schd_empl_cnt / scnd_schd_empl_cnt)
reports['report_4_4'] = f'''
Вакансий самого популярного типа "Полный день — Полная занятость" в {ratio} раз больше,
чем вакансий идущим вторым по популярности типом "Удалённая работа — Полная занятость".
Вывод один: рынку удаленной работы еще расти и расти, а пока будем кататься на работу
утром и домой вечером, сжигая  огромные количества топлива 🔥, электроэнергии ⚡️ и теряя
драгоценное время ⏳ своей жизни в пробках. Но, эволюцию не удержать. Посмотрим на эти
показатели лет так через 10 🤔.
'''

На втором месте по популярности сочетания типа рабочего графика и типа трудоустройства стоит: Удаленная работа—Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [14]:
# текст запроса
query_4_5 = '''
select
	v.experience xp,
	count(v.id) vac_cnt
from vacancies v
group by v.experience
order by 2
'''

# результат запроса
experiences = pd.read_sql_query(query_4_5, connection)
display(experiences)

# визуализация распределения количества вакансий по требуемому опыту
fig = px.bar(
    experiences,
    x='xp', y='vac_cnt', color='vac_cnt',
    title='Распределение количества вакансий по требуемому опыту',
    labels={
		'xp': 'Треубемый опыт',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.update_traces(marker_coloraxis=None)
fig.show()

# вывод отчета об интересном инсайте
six_plus_xp_cnt = experiences.iloc[0]['vac_cnt']
no_xp_cnt = experiences.iloc[1]['vac_cnt']
ratio = round(no_xp_cnt / six_plus_xp_cnt)
reports['report_4_5'] = f'''
Парадоксально 🤔, что вакансий с требованием "Нет опыта" в {ratio} раз больше,
чем 	вакансий с требованием опыта более 6 лет. Можно сделать 3 вывода:
<ol>
<li>Компаний, которые готовы нанимать "дорогих" сотрудников с опытом более 6 лет
меньше на рынке. Скорее всего это крупный бизнес 🧐;</li>
<li>Компании предпочитают обучать людей специфике работы в своем бизнесе, нежели брать
людей с большим опытом работы в других компаниях. Для таким сотрудникам сложнее провести
онбординг, их тяжелее переобучить работать по-новому 🙄;</li>
<li>Руководители компаний не берут опытных сотрудников просто потому, что опасаются 😱
ослабления своих лидерских позиций, за счет проявления большей экспертности таких
новых сотрудников.</li>
</ol>
Так же примечательно, что самая популярная категория опыта работы в вакансиях 1-3 года.
'''

,xp,vac_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [15]:
print_report(reports, 'Выводы по детальному анализу вакансий', 4)

<h1>Раздел проекта: 4. Выводы по детальному анализу вакансий</h1><br>
    <div class='report'>
    <br><h4>
В разрезе распределения ваксансий по регионам следующие инсайты:
<ol>
<li>Москва, ожидаемо, на первом месте 👍🏻</li>
<li>В ТОП-5 регионов кроме столиц (включая историческую) России,
Беларуси и Казахстана вошла научная столица России - Новосибирск.
Поздравим ребят 🥳;</li>
<li>Количество вакансий в Санкт-Петебурге в 2 раза меньше,
чем в Москве. Питер - расти быстрее 💪🏻!</li>
<ol>
</h4><br><h4>
Соотношение количества вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой к
количству вакансий, в которых зарплата не указана вообще: 95.82%.
Получается что практически в половине случаев работодатели не указывают
в объявлении о вакансии ни нижнюю, ни верхнюю границы заработной платы 🙄.
Полагаю, что компании могут делать это с двумя целями:
<ol>
<li>Мотивировать соискателя на взаимодействие с HR-службой через вопрос: "А сколько буду получать 🤔?";</li>
<li>Не раскрывать своей политики в отношении формирования фонда оплаты труда и планов мотивации
конкурентам 🙊.</li>
<ol>
</h4><br><h4>
Верхняя граница заработной платы практически в 1.56 раз выше, чем нижняя 👈🏻.
Можно сделать вывод, что среднестатистически работодатели предполагают возможность
роста заработной платы сотрудника на 56% относительно начальной зарплаты
со временем. Но как известно, средние оценки - просто математический выкрутас,
который больше нацелен на сокращение количества информации, нежели на далеко идущие
выводы 🥸.
</h4><br><h4>
Вакансий самого популярного типа "Полный день — Полная занятость" в 5 раз больше,
чем вакансий идущим вторым по популярности типом "Удалённая работа — Полная занятость".
Вывод один: рынку удаленной работы еще расти и расти, а пока будем кататься на работу
утром и домой вечером, сжигая  огромные количества топлива 🔥, электроэнергии ⚡️ и теряя
драгоценное время ⏳ своей жизни в пробках. Но, эволюцию не удержать. Посмотрим на эти
показатели лет так через 10 🤔.
</h4><br><h4>
Парадоксально 🤔, что вакансий с требованием "Нет опыта" в 5 раз больше,
чем 	вакансий с требованием опыта более 6 лет. Можно сделать 3 вывода:
<ol>
<li>Компаний, которые готовы нанимать "дорогих" сотрудников с опытом более 6 лет
меньше на рынке. Скорее всего это крупный бизнес 🧐;</li>
<li>Компании предпочитают обучать людей специфике работы в своем бизнесе, нежели брать
людей с большим опытом работы в других компаниях. Для таким сотрудникам сложнее провести
онбординг, их тяжелее переобучить работать по-новому 🙄;</li>
<li>Руководители компаний не берут опытных сотрудников просто потому, что опасаются 😱
ослабления своих лидерских позиций, за счет проявления большей экспертности таких
новых сотрудников.</li>
</ol>
Так же примечательно, что самая популярная категория опыта работы в вакансиях 1-3 года.
</h4></div>

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [50]:
# текст запроса
query_5_1 = '''
(select
	e.name employer,
	count(*) vac_cnt
from employers e
	join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
limit 1)
union
(select
	e.name employer,
	count(*) vac_cnt
from employers e
	join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
offset 4 limit 1)
order by 2 desc
'''
# результат запроса
empls = pd.read_sql_query(query_5_1, connection)
print('Это результат буквального выполнения задания')
display(empls)
print('Однако считаю результат малоинформативным. Добавляю немного отсебятины:')

Это результат буквального выполнения задания


,employer,vac_cnt
0,Яндекс,1933
1,Газпром нефть,331


Однако считаю результат малоинформативным. Добавляю немного отсебятины:


In [52]:
# модифицированный запрос
query_5_1_2 = '''
select
	e.name employer,
	count(*) vac_cnt
from employers e
	join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
limit 5
'''

# Результат выполнения запроса
empls = pd.read_sql_query(query_5_1_2, connection)

# визуализация ТОП-5 работодателей по количеству вакансий
# визуализация распределения количества вакансий по требуемому опыту
fig = px.bar(
    empls,
    x='employer', y='vac_cnt', color='employer',
    title='ТОП-5 работодателей по количеству вакансий',
    labels={
		'employer': 'Работодатель',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.update_traces(marker_coloraxis=None)
fig.show()

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [18]:
# текст запроса

In [19]:
# результат запроса

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [20]:
# текст запроса

In [21]:
# результат запроса

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [22]:
# текст запроса

In [23]:
# результат запроса

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [24]:
# текст запроса

In [25]:
# результат запроса

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [26]:
# текст запроса

In [27]:
# результат запроса

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [28]:
# код для получения списка городов-милионников

In [29]:
# текст запроса

In [30]:
# результат запроса

***

In [31]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [32]:
# текст запроса

In [33]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [34]:
# текст запроса

In [35]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [36]:
# текст запроса

In [37]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [38]:
# текст запроса

In [39]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [40]:
# текст запроса

In [41]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [42]:
# текст запроса

In [43]:
# результат запроса

***

In [44]:
# выводы по предметному анализу

# Общий вывод по проекту

In [45]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования